In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Le code de ce cahier a été masqué par défaut pour faciliter la lisibilité
Pour basculer entre les modes, appuyez <a href="javascript:code_toggle()">ici</a>.''')

In [2]:
from IPython.display import Javascript, display, FileLink
from ipywidgets import widgets

def run_all(ev):
    os.remove(filename)
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="(Re) Lancer pyETUDE")
button.on_click(run_all)
display(button)

Button(description='(Re) Lancer pyETUDE', style=ButtonStyle())

In [3]:
import json

with open("pyEtude.json") as json_f:
            jsonData = json.load(json_f)
matieres = jsonData["matieres"]
matieres_list = [f"{mat_name} - {matieres[mat_name][0]}" for mat_name in matieres]
matieres_list_solo = [mat_name for mat_name in matieres]

In [4]:
info_widgets = {}

info_widgets["titre"] = widgets.Text(
    value='',
    placeholder='Les Lois de Newton',
    description='Titre:'
)
info_widgets["sous_titre"] = widgets.Text(
    value='',
    placeholder='Newton, grand physicien',
    description='Sous-Titre:'
)


info_widgets["matiere"] = dict()
info_widgets["matiere"]["drop"] = widgets.Dropdown(
    value="Physique - PHY",
    options=matieres_list,
    description='Matière:',
    disabled=False,
)
info_widgets["matiere"]["check"] = widgets.Checkbox(
    value=False,
    description='Personnaliser:',
    disabled=False,
    indent=True
)
info_widgets["matiere"]["entry"] = widgets.Text(
    value='',
    placeholder='PHY',
    description='',
    disabled=True
)
def checkbox_checked(update):
    state = info_widgets["matiere"]["check"].value
    if state == True:
        info_widgets["matiere"]["drop"].disabled = True
        info_widgets["matiere"]["entry"].disabled = False
    elif state == False:
        info_widgets["matiere"]["drop"].disabled = False
        info_widgets["matiere"]["entry"].disabled = True
info_widgets["matiere"]["check"].observe(checkbox_checked)


info_widgets["numero"] = dict()
info_widgets["numero"]["combo"] = widgets.Combobox(
    value="CHP",
    options=['CHP', 'MOD'],
    description='Numéro:',
    ensure_option=True,
    disabled=False
)
info_widgets["numero"]["intext"] = widgets.BoundedIntText(
    value=1,
    step=1,
    disabled=False
)

info_widgets["section"] = widgets.Text(
    value='',
    placeholder="La Loi de l'Inertie",
    description='1ʳᵉ Section:'
)


info_widgets["generate"] = widgets.Button(
    description='Générer',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Générer le document',
    icon='file' # (FontAwesome names without the `fa-` prefix)
)
def generate_clicked(update):
    info_widgets["titre"].disabled = True
    info_widgets["sous_titre"].disabled = True
    info_widgets["matiere"]["drop"].disabled = True
    info_widgets["matiere"]["check"].disabled = True
    info_widgets["matiere"]["entry"].disabled = True
    info_widgets["numero"]["combo"].disabled = True
    info_widgets["numero"]["intext"].disabled = True
    info_widgets["section"].disabled = True
    info_widgets["generate"].disabled = True
    
    info_widgets["generate"].description = 'Génération...'
    info_widgets["generate"].button_style = 'info'
    info_widgets["generate"].icon = 'ellipsis-h'
    status = generate_document()
    if status == True:
        info_widgets["generate"].description = 'Généré!'
        info_widgets["generate"].button_style = 'success'
        info_widgets["generate"].icon = 'check'
info_widgets["generate"].on_click(generate_clicked)

In [5]:
import os, zipfile, locale
class Document:
    def __init__(self, titre, soustitre, auteur, secondaire, matiere, numero, section, model, filepath):
        self.titre = titre
        self.soustitre = soustitre
        self.auteur = auteur
        self.secondaire = secondaire
        self.matiere = matiere
        self.numero = numero
        self.section = section
        self.model = model
        self.filepath = filepath

        self.options = {"pyETUDE_Titre": titre,
                    "pyETUDE_SousTitre": soustitre,
                    "pyETUDE_Matiere": matiere,
                    "pyETUDE_Auteur": auteur,
                    "pyETUDE_Sec": secondaire,
                    "pyETUDE_Num": numero}
        self.sections = {"sectionpy": section}

        self.folder = f"{matiere}-{numero}_tmpyETUDE"

        self.main()

    def main(self):
        self.exportWord(self.model, self.folder)

        self.modifyOptions(os.path.join(self.folder, "word", "document.xml"), self.options)
        self.modifyOptions(os.path.join(self.folder, "word", "header1.xml"), self.options)
        self.modifyOptions(os.path.join(self.folder, "word", "footer1.xml"), self.options)

        self.modifyOptions(os.path.join(self.folder, "word", "document.xml"), self.sections)

        self.packWord(self.folder, self.filepath)
        self.cleanTemp(self.folder)

    def exportWord(self, model:str, folder:str) -> str:
        """## Extract the specified `.zip` file

        ### Arguments:\n
            \tmodel {str} -- file that will be extracted (Do not forget the .docx!)
            \tfolder {str} -- folder that will receive the extracted file

        ### Returns:\n
            \tstr -- the name of the folder where it was extracted
        """

        with zipfile.ZipFile(model, "r") as model_file:
            model_file.extractall(folder)
            model_file.close()
        return folder

    def modifyOptions(self, path:str, info:dict) -> str:
        """## Send command to {self.searchAndReplace} for all values in a dictionnary
        
        ### Arguments:\n
            \tpath {str} -- The path of the file to be modified
            \tinfo {dict} -- A dictionnary in this format {"to search":"to replace"}
        
        ### Returns:\n
            \tstr -- Returns the name of the file modified
        """
        for search, replace in info.items():
            self.searchAndReplace(path, search, replace)
        return path

    def searchAndReplace(self, infile:str, search:str, replace:str) -> str:
        """## Search the specified file with the keyword given and replaces it with the third argument
        
        ### Arguments:\n
            \tinfile {str} -- The file to change
            \tsearch {str} -- The word to replace
            \treplace {str} -- The word that will be replaced by {search}

        ### Returns:\n
            \tstr -- Returns the name of the file modified
        """
        if os.path.isfile(infile):
            with open(infile, "r", encoding="utf8") as in_f:
                data_f = in_f.read()
                with open(infile, "w", encoding='utf8') as out_f:
                    out_f.write(data_f.replace(search, replace))
        else:
            raise FileNotFoundError
        return infile


    def packWord(self, folder:str, final:str) -> str:
        """## Zip the folder specified
        This will only zip the contents of the folder, not the base folder

        ### Arguments:\n
            \tfolder {str} -- the folder that will be zipped
            \tfinal {str} -- the name of the archive (Do not forget the .docx!)

        ### Returns:\n
            \tstr -- the name of the zip file that was created
        """
        locale.setlocale(locale.LC_ALL, (None, None))  # Fix compatibility with locale
        with zipfile.ZipFile(final, "w", compression=zipfile.ZIP_DEFLATED) as zip_file:
            for root, dirs, files in os.walk(folder):  # pylint: disable=unused-variable
                # zip_file.write(os.path.join(root, "."))

                for File in files:
                    filePath = os.path.join(root, File)
                    inZipPath = filePath.replace(folder, "", 1).lstrip("\\/")
                    zip_file.write(filePath, inZipPath)
        #print(f"\nLe document a été créé: {self.filepath}")  # Si démarré à partir de l'invite de commande

        return final

    def cleanTemp(self, folder:str) -> str:
        """## Clean the temporary folder
        DANGEROUS: THIS WILL DELETE ALL THE FILES IN THE SPECIFIED FOLDER!!
        
        ### Arguments:\n
            \tfolder {str} -- The folder that will be deleted
        
        ### Raises:\n
            \tNotADirectoryError: The specified folder is not a folder
        
        ### Returns:\n
            \tstr -- Returns the name of the folder deleted
        """
        if os.path.isdir(folder):
            for root, dirs, files in os.walk(folder, topdown=False):
                for File in files:
                    os.remove(os.path.join(root, File))
                for Dir in dirs:
                    os.rmdir(os.path.join(root, Dir))
            os.rmdir(folder)
        else:
            raise NotADirectoryError
        
        return folder

In [6]:
def generate_document():
    global jsonData, matieres, filename
    titre = info_widgets["titre"].value
    if titre == "":
        titre = info_widgets["titre"].placeholder
    
    sous_titre = info_widgets["sous_titre"].value
    if sous_titre == "":
        sous_titre = info_widgets["sous_titre"].placeholder
        
    if info_widgets["matiere"]["drop"].disabled == False:
        matiere = matieres[matieres_list_solo[info_widgets["matiere"]["drop"].index]][0]
    else:
        matiere = info_widgets["matiere"]["entry"].value
        if matiere == "":
            matiere = info_widgets["matiere"]["entry"].placeholder
    
    numero = info_widgets["numero"]["combo"].value + str(info_widgets["numero"]["intext"].value)
    filename = f"{matiere}-{numero}.docx"
    
    section = info_widgets["section"].value
    if section == "":
        section = info_widgets["section"].placeholder
    
    auteur = jsonData["auteur"]
    secondaire = jsonData["secondaire"]
    
    model = "model.docx"
    
    filepath = os.path.join(os.getcwd(), filename)
    
    Document(titre, sous_titre, auteur, secondaire, matiere, numero, section, model, filepath)
    
    display(FileLink(filename, result_html_prefix="Cliquez ici pour télécharger votre document généré: "))
    return True

In [7]:
# Invisible space ("braille"): widgets.Label("⠀")
widgets.GridBox((info_widgets["titre"], info_widgets["sous_titre"], widgets.Label("⠀"),
             widgets.Label("⠀"), widgets.Label("⠀"), widgets.Label("⠀"),
             info_widgets["matiere"]["drop"], info_widgets["matiere"]["check"], info_widgets["matiere"]["entry"],
             info_widgets["numero"]["combo"], info_widgets["numero"]["intext"], widgets.Label("⠀"),
             widgets.Label("⠀"), widgets.Label("⠀"), widgets.Label("⠀"),
             info_widgets["section"], widgets.Label("⠀"), widgets.Label("⠀"),
             widgets.Label("⠀"), widgets.Label("⠀"), widgets.Label("⠀"),
             info_widgets["generate"]),
               layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

GridBox(children=(Text(value='', description='Titre:', placeholder='Les Lois de Newton'), Text(value='', descr…

C:\Users\Laurent\GitHub\pyEtude\PHY-CHP1.docx